# Querying the data lake using natural language

This notebook will connect an LLM to our data lake to help answer questions.

## Configure variables

In [3]:
region = 'us-east-1'
s3stagingathena=f'<athena s3 staging area>'

connathena=f"athena.{region}.amazonaws.com"
portathena='443' 
schemaathena='default' 
wkgrpathena='primary'

In [1]:
## Install our prereqs
%pip install \
    "boto3" \
    "awscli" \
    "botocore" \
    "langchain" \
    "langchain_experimental" \
    "sqlalchemy" \
    "PyAthena"

    

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 15.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 78.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 120.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import boto3
import langchain
from langchain.llms.bedrock import Bedrock

boto3_bedrock = boto3.client(
            service_name="bedrock-runtime"
)

modelId = 'anthropic.claude-v2:1'
llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    streaming=True,
    model_kwargs={"max_tokens_to_sample": 2048, 'temperature': 0.75},
)


In [9]:
# Athena Setup
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine

connection_string = f"awsathena+rest://@{connathena}:{portathena}/{schemaathena}?s3_staging_dir={s3stagingathena}/&work_group={wkgrpathena}"
engine_athena = create_engine(connection_string, echo=False)
db = SQLDatabase(engine_athena)


In [10]:
# setup tools and agents
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [12]:
agent_executor.run("How many reviews came in during the month of October 2014 where the overall score was 3 or less?")



> Entering new AgentExecutor chain...
 Let's break this down step-by-step:

Action: sql_db_list_tables
Action Input: 
Observation: diamonds, fla_us_east_2, ny_taxi_s3, reviews, tae
Thought: Here is my step-by-step approach to answering the question:

Action: sql_db_list_tables
Action Input:  
Observation: diamonds, fla_us_east_2, ny_taxi_s3, reviews, tae
Thought: Here is the step-by-step process to answer the question:

Action: sql_db_list_tables
Action Input:  
Observation: diamonds, fla_us_east_2, ny_taxi_s3, reviews, tae
Thought: Here is the step-by-step process to answer the question:

Action: sql_db_schema  
Action Input: reviews
Observation: 
CREATE EXTERNAL TABLE reviews (
	overall DOUBLE,
	vote STRING,
	verified BOOLEAN,
	asin STRING,
	style STRING,
	reviewtext STRING,
	summary STRING,
	tsreviewtime TIMESTAMP
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
WITH SERDEPROPERTIES (
	'serialization.format' = '1'
)
STORED AS INPUTFORMAT 'org.apache

'599,787 reviews came in during the month of October 2014 where the overall score was 3 or less'

In [13]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

In [14]:
agent_executor.run("Find reviews from October 2014 with an overall score of 1 or less. Then pick five examples and summarize the review text.")



> Entering new AgentExecutor chain...
 Sounds good, here are the steps I will take:

Action: sql_db_list_tables
Action Input: 

Observation: diamonds, fla_us_east_2, ny_taxi_s3, reviews, tae
Thought: Based on the question, it seems like the "reviews" table would be the most relevant to query. I will check the schema for that table to understand what columns are available.

Action: sql_db_schema
Action Input: reviews

Observation: 
CREATE EXTERNAL TABLE reviews (
	overall DOUBLE,
	vote STRING,
	verified BOOLEAN,
	asin STRING,
	style STRING,
	reviewtext STRING,
	summary STRING,
	tsreviewtime TIMESTAMP
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
WITH SERDEPROPERTIES (
	'serialization.format' = '1'
)
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION 's3://genaidatalake-s3bucket6575f0a6-dgaafbxi58dw/data/parquet/'
TBLPROPERTIES (
	

"Here is a summary of the 5 review examples with overall score 1 or less from October 2014:\n\n1) The clothing item was a strange, overly large size compared to what was advertised. \n\n2) The shoes were too tight and small compared to typical sizing for that brand.\n\n3) The work boots did not meet the customer's expectations and took too long to dry.  \n\n4) The product stopped blowing warm air after only a week - it broke very quickly.\n\n5) The clothing item was much too small for an average build customer.\n\nIn summary, these negative reviews mainly complained about sizing issues, products not meeting expectations, and quick breakage. Many felt the products were not true to advertised size or their typical experience with that brand."

In [15]:
agent_executor.run("Summarize the customer reviews for asin B005V7U52A?")



> Entering new AgentExecutor chain...
 Sounds good, here is how I would approach this:

Action: sql_db_list_tables
Action Input: 
Observation: diamonds, fla_us_east_2, ny_taxi_s3, reviews, tae
Thought: Here is how I would continue solving this step-by-step:

Action: sql_db_schema
Action Input: reviews
Observation: 
CREATE EXTERNAL TABLE reviews (
	overall DOUBLE,
	vote STRING,
	verified BOOLEAN,
	asin STRING,
	style STRING,
	reviewtext STRING,
	summary STRING,
	tsreviewtime TIMESTAMP
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
WITH SERDEPROPERTIES (
	'serialization.format' = '1'
)
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION 's3://genaidatalake-s3bucket6575f0a6-dgaafbxi58dw/data/parquet/'
TBLPROPERTIES (
	'EXTERNAL' = 'TRUE',
	'inputformat' = 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat',
	'location' = '

'The product with asin B005V7U52A has an average customer rating of 4.0 stars based on 26 reviews.'